In [58]:
from sklearn.neighbors import KNeighborsClassifier

import numpy as np

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

path = '/workspace/data/crops_and_embeddings/martha_loango/embeddings'

embeddings = np.load(f'{path}/embeddings.npy')
labels = np.load(f'{path}/individual_ids.npy')

assert len(embeddings) == len(labels)

print(embeddings.shape)
print(labels.shape)

print(len(set(labels)))

(138, 256)
(138,)
{'TONDA', 'EVOUNDE', 'MOUTCHI', 'IPIKLIA', 'ROBBINS', 'IPIKILIA', 'EVOANDE', 'AMBIA', 'OWABI', 'DIBUTI', 'ELIWA', 'MOKABO', 'WAKA', 'OWEGELY', 'MALUMBI', 'OREMA', 'MOKEBO', 'OZAVINO', 'TONDAY', 'OGWELY', 'KAMAYA', 'EVOUANDE'}


In [57]:
def predict_one(embeddings, labels, test_idx, k=5):
    train_idx = [i for i in range(len(embeddings)) if i != test_idx]
    train_embeddings = embeddings[train_idx]
    train_labels = labels[train_idx]

    test_embedding = embeddings[test_idx]

    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(train_embeddings, train_labels)

    return knn.predict([test_embedding])[0], labels[test_idx]

# predict_one(embeddings, labels, 3)

# Calculate precision, recall, f1

def calculate_metrics(embeddings, labels, min_samples=5, k=2):
    # only predicting on individuals with at least min_samples
    unique_labels, counts = np.unique(labels, return_counts=True)
    unique_labels = unique_labels[counts >= min_samples]
    
    mask = np.isin(labels, unique_labels)
    # embeddings = embeddings[mask]
    # labels = labels[mask]
    
    idx = [i for i in range(len(embeddings)) if labels[i] in unique_labels]
    
    print(f'Calculating metrics on {len(idx)} samples from {len(unique_labels)} individuals with {len(embeddings)} samples in total.')
    
    predictions = [predict_one(embeddings, labels, i, k=k) for i in idx]
    predictions = np.array(predictions)
    
    tp = np.sum(predictions[:, 0] == predictions[:, 1])
    
    average_precision =  tp / len(embeddings)
    
    return average_precision
 

print(calculate_metrics(embeddings, labels, min_samples=1, k=3))
print(calculate_metrics(embeddings, labels, min_samples=5, k=3))

Calculating metrics on 138 samples from 22 individuals with 138 samples in total.
0.41304347826086957
Calculating metrics on 119 samples from 10 individuals with 138 samples in total.
0.4057971014492754


In [ ]:
Calculating metrics on 138 samples from 22 individuals with 138 samples in total.
0.36231884057971014
Calculating metrics on 119 samples from 10 individuals with 138 samples in total.
0.36231884057971014